In [1]:
import sqlite3
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import timedelta
from pandas.core.groupby.generic import DataFrameGroupBy
from icecream import ic
import re
import sys
sys.path.append("..")
from src.fpl.pipelines.model_pipeline.preprocessing.elo_calculation import calculate_elo_score

In [5]:
conn = sqlite3.connect("../data/fpl.db")
match_log = pd.read_sql(f"select * from raw_team_match_log", conn)
conn.close()
display(match_log.head())

,season,team,date,time,comp,round,day,venue,result,gf,ga,opponent,xg,xga,poss,link
0,2021-2022,Manchester City,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0.0,1.0,Leicester City,NaN,NaN,57.0,https://fbref.com/en/squads/b8fd03ef/2021-2022...
1,2021-2022,Manchester City,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,2.0,1.0,65.0,https://fbref.com/en/squads/b8fd03ef/2021-2022...
2,2021-2022,Manchester City,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,2.7,0.1,67.0,https://fbref.com/en/squads/b8fd03ef/2021-2022...
3,2021-2022,Manchester City,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,4.0,0.2,80.0,https://fbref.com/en/squads/b8fd03ef/2021-2022...
4,2021-2022,Manchester City,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,3.3,0.6,61.0,https://fbref.com/en/squads/b8fd03ef/2021-2022...


In [3]:
data = match_log.copy()
parameters = dict(
    elo_learning_rate=0.1, 
    home_away_weight=0.5,
    start_year="2016-2017",
    use_cache=False
    )
elo_data = calculate_elo_score(data, None, parameters)
elo_data

100%|██████████| 3061/3061 [00:18<00:00, 165.23it/s]


,team,season,date,att_elo,def_elo,home_att_elo,home_def_elo,away_att_elo,away_def_elo
0,Arsenal,2016-2017,2016-08-12,NaN,NaN,NaN,NaN,NaN,NaN
1,Bournemouth,2016-2017,2016-08-12,NaN,NaN,NaN,NaN,NaN,NaN
2,Burnley,2016-2017,2016-08-12,NaN,NaN,NaN,NaN,NaN,NaN
3,Chelsea,2016-2017,2016-08-12,NaN,NaN,NaN,NaN,NaN,NaN
4,Crystal Palace,2016-2017,2016-08-12,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6137,Fulham,2023-2024,2024-05-19,NaN,NaN,NaN,NaN,NaN,NaN
6138,Manchester City,2023-2024,2024-05-19,NaN,NaN,NaN,NaN,NaN,NaN
6139,West Ham,2023-2024,2024-05-19,NaN,NaN,NaN,NaN,NaN,NaN
6140,Sheffield Utd,2023-2024,2024-05-19,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data[(data["ast_ma1"].isna()) & (data["round"]==6)]

KeyError: 'ast_ma1'